## prep address



In [1]:
import os
import pandas as pd

In [17]:
# Указать путь к папке, содержащей файлы .xlsx
folder_path = "../prep_data/"

# Список для хранения pandas DataFrame
dataframes = []

# Чтение файлов .xlsx в указанной папке
for file_name in os.listdir(folder_path):
    if file_name.endswith(".xlsx") and file_name.startswith(("1", "2")):
        file_path = os.path.join(folder_path, file_name)
        print(file_name)
        # Чтение файла в DataFrame
        df = pd.read_excel(file_path, header=None)
        df[11] = pd.to_datetime(df[11], errors="coerce")
        df.dropna(subset=[11], inplace=True)
        df.reset_index(drop=True, inplace=True)
        # Добавление DataFrame в список

        df[11] = pd.to_datetime(df[11])
        df["formatted_date"] = df[11].dt.strftime("%m.%Y")
        df.drop(columns=[0, 1, 5, 8, 10], inplace=True)
        df.columns = [
            "smeta",
            "distr",
            "code",
            "code_discription",
            "address_raw",
            "cost",
            "full_date",
            "data_short",
        ]
        df.to_excel("./data/" + file_name)

In [40]:
def to_lower(listin):
    return [str(element).lower() for element in listin]


citys = pd.read_excel("./np-dvfo.xlsx", header=None, engine="openpyxl")[0].to_list()
streets = pd.read_excel("./names-dvfo.xlsx", header=None, engine="openpyxl")[
    0
].to_list()
numbers = [str(i) for i in range(1, 1000)]
citys = to_lower(citys)
streets = to_lower(streets)
numbers = to_lower(numbers)

In [44]:
import re

folder_path = "../prep_data/"
# Чтение файлов .xlsx в указанной папке
for file_name in os.listdir(folder_path):
    if file_name.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file_name)

        # Чтение файла в DataFrame
        df = pd.read_excel(file_path)
        df["address"] = df["address"].str.lower()
        df["address_new"] = df["address"].str[-30:]
        df["address_new"] = df["address_new"].apply(
            lambda x: re.sub(r"[^\w\s]", " ", x)
        )

        # Удаление старой колонки
        df.drop(columns=["address"], axis=1, inplace=True)

        # Шаг 2: Оставить только слова и последовательности цифр не длинее 3 знаков подряд
        # df['test'] = df['test'].apply(lambda x: re.sub(r'\b(?:\d{1,3}|\w{1,})\b', ' ', x))

        # Шаг 3: Оставить только те слова, которые лежат в одном из двух списков: cities, streets
        df["address_new"] = df["address_new"].apply(
            lambda x: " ".join(
                word
                for word in x.split()
                if word in citys or word in streets or word in numbers
            )
        )
        df.to_excel("../prep_data/out/" + "clean_" + file_name, index=False)